In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(3).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [10]:
history = model.fit(train_data,
                    epochs=25,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 10:56:31.959747: {'loss': 2.070271674378713, 'accuracy': 0.47959167, 'val_loss': 8.244375947174753, 'val_accuracy': 0.3748}
Epoch 1 ends at 10:59:23.289843: {'loss': 1.0668118411699932, 'accuracy': 0.6682917, 'val_loss': 0.8808854036270433, 'val_accuracy': 0.706}
Epoch 2 ends at 11:02:19.281338: {'loss': 0.661705203962326, 'accuracy': 0.77391666, 'val_loss': 0.7410448930066103, 'val_accuracy': 0.7564}
Epoch 3 ends at 11:05:21.429058: {'loss': 0.4523838563323021, 'accuracy': 0.8408167, 'val_loss': 0.7208153119512425, 'val_accuracy': 0.7736}
Epoch 4 ends at 11:08:28.874187: {'loss': 0.3150237764557203, 'accuracy': 0.8886333, 'val_loss': 0.9760374100344955, 'val_accuracy': 0.7524}
Epoch 5 ends at 11:11:36.963089: {'loss': 0.2182220031698545, 'accuracy': 0.92385, 'val_loss': 0.8356989789160953, 'val_accuracy': 0.7955}
Epoch 6 ends at 11:14:44.863552: {'loss': 0.17213616901536782, 'accuracy': 0.93934166, 'val_loss': 0.9421750396300274, 'val_accuracy': 0.7559}
Epoch 7 ends at

In [11]:
model.evaluate(test_data)

157/157 [==============================]1.1396 - accuracy: 0.828 - 0s 81ms/step - loss: 1.1606 - accuracy: 0.804 - 0s 70ms/step - loss: 1.2450 - accuracy: 0.81 - 0s 64ms/step - loss: 1.1202 - accuracy: 0.82 - 0s 60ms/step - loss: 1.0218 - accuracy: 0.82 - 0s 57ms/step - loss: 1.0419 - accuracy: 0.83 - 0s 58ms/step - loss: 1.0473 - accuracy: 0.83 - 0s 55ms/step - loss: 1.1477 - accuracy: 0.81 - 0s 53ms/step - loss: 1.0762 - accuracy: 0.82 - 1s 53ms/step - loss: 1.1242 - accuracy: 0.81 - 1s 52ms/step - loss: 1.0555 - accuracy: 0.82 - 1s 51ms/step - loss: 1.0374 - accuracy: 0.82 - 1s 51ms/step - loss: 1.0324 - accuracy: 0.82 - 1s 50ms/step - loss: 1.0378 - accuracy: 0.82 - 1s 50ms/step - loss: 1.0236 - accuracy: 0.82 - 1s 49ms/step - loss: 1.0568 - accuracy: 0.82 - 1s 49ms/step - loss: 1.0517 - accuracy: 0.82 - 1s 49ms/step - loss: 1.0766 - accuracy: 0.81 - 1s 48ms/step - loss: 1.0650 - accuracy: 0.81 - 1s 48ms/step - loss: 1.0866 - accuracy: 0.81 - 1s 48ms/step - loss: 1.0589 - accuracy:

[1.1201782190495995, 0.813]

In [12]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))